In [1]:
from model.rnn import GRUDecoder
import torch        

In [2]:
def get_device():
    # Check if CUDA is available
    if torch.cuda.is_available():
        # If CUDA is available, select the first CUDA device
        device = torch.device("cuda:0")
        print("Using CUDA device:", torch.cuda.get_device_name(0))
    # Check for MPS availability on supported macOS devices (requires PyTorch 1.12 or newer)
    elif torch.backends.mps.is_available():
        # If MPS is available, use MPS device
        device = torch.device("mps")
        print("Using MPS (Metal Performance Shaders) device")
    else:
        # Fallback to CPU if neither CUDA nor MPS is available
        device = torch.device("cpu")
        print("Using CPU")
    return device


In [3]:
model = GRUDecoder(input_size=12, hidden_size=10).to(get_device())

Using MPS (Metal Performance Shaders) device


In [4]:
x = torch.randn(4,10,12).to(get_device())

Using MPS (Metal Performance Shaders) device


In [7]:
xx, _ = model.gru(x)
xx = model.post_gru(xx)

In [9]:
model.attention(xx)

RuntimeError: einsum(): the number of subscripts in the equation (2) does not match the number of dimensions (1) for operand 1 and no ellipsis was given

In [11]:
model.attention.context.shape

torch.Size([10])

In [17]:
scores = torch.einsum("bij, j -> bi", xx, model.attention.context)
scores.shape

torch.Size([4, 10])

In [19]:
torch.einsum("bij, bj -> bi", xx, scores)

torch.Size([4, 10])